# Get All Tweets with #podrevday

In [ ]:
!mkdir data

In [ ]:
!GetOldTweets3 --querysearch "podrevday" --since 2020-01-01 --until 2020-07-10 --output "data/jan-july-2020.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()

import twint

df = pd.read_csv('data/jan-july-2020.csv', parse_dates=['date'])

# Get User Data

In [ ]:
pod_rev_users = list(set(df.username))

c = twint.Config()
c.Store_object = True
c.Pandas = True

for user in pod_rev_users: 
    c.Username = user
    twint.run.Lookup(c)
Users_df = twint.storage.panda.User_df

users_df = Users_df.drop_duplicates()
users_df.to_csv('data/user_data.csv')